In [529]:
#import packages
import numpy as np
import pandas as pd
import cv2
import time
cv2.startWindowThread()


0

In [530]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Daniel_7_top_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps

if cap.isOpened() == False:
    print("Cannot open video file.")

In [531]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
#print(size)
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (1700,700))
#output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

In [532]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [533]:
#creating empty array
# Create an empty DataFrame with columns for x and y velocities
velocities = pd.DataFrame(columns=['x_velocity', 'y_velocity'])
prev_x, prev_y = None, None
prev_time = None


In [534]:
#defining rgb range

lower_color_bound = np.array([0,100,178])
upper_color_bound = np.array([255,255,255])

In [535]:
#conversion factor
disc_length = 8.26 #disc length in inches
distance = 36 #distance of disc from camera, rough guess

# Approximate field of view (in degrees)
fov = 2 * np.arctan(frame_width / (2 * distance)) * 180 / np.pi
print(fov)

# Width of the object in pixels
object_width_pixels = abs((disc_length/2) / (np.tan(fov/2)))
print (object_width_pixels)
#calculate conversion rate
conversion_rate = disc_length / object_width_pixels
print(conversion_rate)



175.70482914340297
36.661356887239556
0.22530535422912829


In [536]:
while True:
    # Get current time
    current_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break
    #regions of interest
    height, width, _ = frame.shape
    roi = frame[200:900, 100:1800]
    
    #object detection
    mask = object_detector.apply(roi)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #trying rgb threshold
    mask = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV) 
    mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    

    for cnt in rgb_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(roi, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            #print(x,y,w,h)
            rect_coordinates = np.append(rect_coordinates,[x,y,w,h])
            #rect_coordinates[y:y+h, x:x+w] = 1
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0,255,0),3)
    #print(rect_coordinates)
    
    # Check if previous position and time are available
    if prev_x is not None and prev_y is not None and prev_time is not None:
        # Calculate the distance traveled in the x and y directions
        dx = x - prev_x
        dy = y - prev_y

        # Calculate the time interval
        dt = current_time - prev_time

        # Calculate the velocity in the x and y directions
        vx = dx / dt
        vy = dy / dt

        

        # Add the velocity values to the DataFrame
        velocities = pd.concat([velocities, new_data])

        vx_mph = vx * conversion_rate * 0.44704
        vy_mph = vy * conversion_rate * 0.44704

        # create a new dataframe with the new velocities
        new_data = pd.DataFrame({'x_velocity': [vx_mph], 'y_velocity': [vy_mph]})
        
        # Print the velocity values
        # print("Velocity in x:", vx, "pixels/s, Velocity in y:", vy, "pixels/s")
        #print("Velocity in x:", vx_mph, "mph, Velocity in y:", vy_mph, "mph")

    # Update the previous position and time variables
    prev_x, prev_y = x, y
    prev_time = current_time
    
    cv2.imshow("roi", roi)
    cv2.imshow("Mask", mask)
    cv2.imshow("maskrgb",mask_rgb)
    
    #output.write(roi)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break


cap.release()
output.release()
#output2.release()
cv2.destroyAllWindows()

In [537]:
velocities.to_csv("velocities.csv", index=False)
print(velocities)

    x_velocity  y_velocity
0    -0.000000   -0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0 -1170.901779  -68.465617
0     0.000000  -21.134738
0     7.067755   21.203264
0    47.454097   16.748505
0    53.677197    7.156960
0    53.313900    2.077165
0    55.266607    0.690833
0    57.932811    1.412995
0    56.345682    2.012346
0    59.008909    0.000000
0    58.183824    0.701010
0    58.011913    0.000000
0    58.995359    1.404651
0    58.356978    1.389452
0    58.443740    1.359157
0    59.469077    0.000000
0    58.400004    1.390476
0    63.751203    1.448891
0    62.396463    0.709051
0    59.965244    0.681423
0    63.498582    1.459738
0    62.371002    1.433816
0    61.498279   27.173658
0     0.000000    0.000000
0     0.000000    0.000000
0